In [1]:
import GPflow
import numpy as np
from matplotlib import pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

ImportError: No module named GPflow

In [18]:
import csv
import time
import datetime
from dateutil import parser
import collections

lookup = {}
with open('uber-raw-data-apr14.csv', 'rb') as f:
    reader = csv.reader(f)
    for row in reader:
        times = row[0][:-5]+'00:00'
        d = parser.parse(times)
        timestamp = int(time.mktime(d.timetuple()))
        lat = row[1]
        lon = row[2]
        
        if timestamp in lookup:
            lookup[timestamp] += 1
        else:
            lookup[timestamp] = 1

od = collections.OrderedDict(sorted(lookup.items()))
with open('result.csv', 'wb') as csv_file:
    writer = csv.writer(csv_file)
    i = 1;
    for key, value in od.items():
        writer.writerow([datetime.datetime.fromtimestamp(int(key)).strftime('%Y-%m-%d %H:%M:%S'),key,i, value])
        i+=1

In [ ]:
y=np.array([2.0e4, 2.8e4, 3.2e4, 3.8e4, 4.2e4, 4.0e4, 3.0e4, 3.0e4]).astype('f')
y0 = y/y.max()
x = np.arange(1,1+len(y)).astype('f')
plt.scatter(x,y0)

In [ ]:
k = GPflow.kernels.PeriodicKernel(1, period=7.0)
m = GPflow.gpr.GPR(x[:,None], y0[:,None], kern=k)
m.likelihood.variance = 0.01

In [ ]:
def plot(m):
    xx = np.linspace(x.min()-1, x.max()+1, 100)[:,None]
    mean, var = m.predict_y(xx)
    plt.figure(figsize=(12, 6))
    plt.plot(x, y0, 'kx', mew=2)
    plt.plot(xx, mean, 'b', lw=2)
    plt.fill_between(xx[:,0], mean[:,0] - 2*np.sqrt(var[:,0]), mean[:,0] + 2*np.sqrt(var[:,0]), color='blue', alpha=0.2)
plot(m)

In [ ]:
k_daily = GPflow.kernels.PeriodicKernel(1, period=24.0)
k_weekly = GPflow.kernels.PeriodicKernel(1, period=24.0*7.0)
k_growth = GPflow.kernels.Linear(1)
k_total = k_daily+k_weekly+k_growth

In [ ]:
k_total